In [104]:
import string
import re
import math 
import numpy as np
from collections import Counter
# -*- coding: <utf-8> -*-


#The function below calculates bigrams for the trainig data after doing some text processing.  
def bigrams_training(x):
    a = open(x).read()
    b = a.replace("\\'", "") 
    c = b.lower() #making everything lower case
    exclude = set(string.punctuation)
    d = ''.join(ch for ch in c if ch not in exclude)
    e =  d.replace("\xad", "") 
    f = re.sub(r'\d+', '', e)
    g = re.sub('\n', ' ', f)
    bigrams = Counter(zip(g,g[2:]))
    return bigrams  

#The function below calculates unigrams for the training data after doing some text processing.  
def unigrams_training(x):
    a = open(x).read()
    b = a.replace("\'", "") 
    c= b.lower() #making everything lower case
    exclude = set(string.punctuation)
    d = ''.join(ch for ch in c if ch not in exclude)
    e =  d.replace("\xad", "") 
    f = re.sub(r'\d+', '', e)
    g = re.sub('\n', ' ', f)
    unigrams = Counter(zip(g))
    return unigrams

#The function below calculates the MLE for the training data.  
def probs_training(bigrams, unigrams):
    d = {}
    for k_b, v_b in bigrams.items():
        for  k_u, v_u in unigrams.items():
            if k_b[0] in k_u:
                prob = v_b/v_u
                d[k_b] = prob
    return d

#the function below calculates bigrams for the test data after doing some text processing. .
def bigrams_test():
    test_bi = []
    with open('LangId.test') as f:
        #content = f.readlines()
        for line in f:
            line = line.lower()
            exclude = set(string.punctuation)
            d = ''.join(ch for ch in line if ch not in exclude)
            e =  d.replace("\xad", "") 
            f = re.sub(r'\d+', '', e)
            g = re.sub('\n', ' ', f)
            bigrams = Counter(zip(g,g[2:]))
            test_bi.append(list(bigrams.keys()))
    return test_bi

        

#The variables below assign different probability models for each language.
eng_bigrams = bigrams_training('LangId.train.English')
eng_unigrams = unigrams_training('LangId.train.English')
probs_eng = probs_training(eng_bigrams, eng_unigrams)


fr_bigrams = bigrams_training('LangId.train.French')
fr_unigrams = unigrams_training('LangId.train.French')
probs_fr = probs_training(fr_bigrams, fr_unigrams)


it_bigrams = bigrams_training('LangId.train.Italian')
it_unigrams = unigrams_training('LangId.train.Italian')
probs_it = probs_training(it_bigrams, it_unigrams)

#The variable below creates the bigrams for the test data. 
test_bigrams = bigrams_test()  

#The function below evaluates each model and applies add-one smoothing. 
def probs_test():
    log_probs = []
    eval_probs = []
    for line_no, line in enumerate(test_bigrams):
        eval_probs.append({'fr': [] , 'en': [], 'it': []})
        for bigram in line:
            if bigram in probs_eng:
                eval_probs[line_no]['en'].append(math.log(probs_eng[bigram]))
            else:
                eval_probs[line_no]['en'].append(1.0)
            if bigram in probs_fr:
                eval_probs[line_no]['fr'].append(math.log(probs_fr[bigram]))
            else:
                eval_probs[line_no]['fr'].append(1.0)
            if bigram in probs_it:
                eval_probs[line_no]['it'].append(math.log(probs_it[bigram]))
            else:
                eval_probs[line_no]['it'].append(1.0)
                                                 
        log_probs.append((sum(eval_probs[line_no]['en']), 
                         sum(eval_probs[line_no]['fr']), 
                          sum(eval_probs[line_no]['it'])))
                                                 
    return log_probs

#The code below does some simple data processing to save the file in a friendly format for the reader. 
import numpy as np
test_probs = probs_test()
np_test_probs = np.array(test_probs)
classification = np.argmax(np_test_probs, axis=1)
demux = ['English', 'French', 'Italian']
result = [demux[i] for i in classification]
#print(result)

l = []
for i in range(1, 301):
    l.append(i)


import csv
with open('letterLangId.out.txt', 'w') as f:
    writer = csv.writer(f, delimiter=' ')
    writer.writerows(zip(l,result))

In [113]:
#The code below gives me how many times I got the language correctly. 
with open('LangId.sol', 'r') as myfile:
    data=myfile.read()
for i in data:
     w = re.findall('\w+', data)
no_integers = [x for x in w if not (x.isdigit() 
                                         or x[0] == '-' and x[1:].isdigit())]
import itertools

len([i for i, j in zip(result, no_integers) if i == j])



        



229